# Neural network basics
----

## Given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?
The case study is from an open source dataset from Kaggle.

Link to the Kaggle project site:
https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Import libraries

In [0]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix

### Read the dataset

In [0]:
ds = pd.read_csv("Churn_Modelling.csv")

In [4]:
ds.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


### Drop the columns which are unique for all users like IDs

In [5]:
ds['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

### RowNumber, CustomerId, and Surname are unique hence we are dropping it

In [0]:
ds = ds.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [7]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


### Distinguish the feature and target set

In [0]:
X = ds.iloc[:,0:10].values # Credit Score through Estimated Salary
y = ds.iloc[:,10].values # Exited

### Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany. This will convert those strings into scalar values for analysis.

In [9]:
print(X[:8,1], '... will now become: ')

label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


### We will do the same thing for gender. this will be binary in this dataset

In [10]:
print(X[:6,2], '... will now become: ')

label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


### The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). Therefore, one way to get rid of that problem is to split the countries into respective dimensions. Gender does not need this as it is binary

### Converting the string features into their own dimensions. Gender doesn't matter here because its binary

In [11]:
countryhotencoder = OneHotEncoder(categorical_features = [1]) # 1 is the country column
X = countryhotencoder.fit_transform(X).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [12]:
X.shape

(10000, 12)

In [13]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [0]:
# A 0 on two countries means that the country has to be the one variable which wasn't included 
# This will save us from the problem of using too many dimensions
X = X[:,1:] # Got rid of Spain as a dimension.

### Divide the data set into Train and test sets

In [0]:
# Splitting the dataset into the Training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

### Normalize the train and test data

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Initialize & build the model

In [0]:
# Initializing the ANN
classifier = Sequential()

In [0]:
# The amount of nodes (dimensions) in hidden layer should be the average of input and output layers, in this case 6.
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [0]:
#Add 1st hidden layer
classifier.add(Dense(6, activation='sigmoid', kernel_initializer='uniform'))

In [0]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [0]:
# Create optimizer with default learning rate
# sgd_optimizer = tf.keras.optimizers.SGD()
# Compile the model
classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [22]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [23]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 2s 192us/sample - loss: 0.2120 - accuracy: 0.7620 - val_loss: 0.1844 - val_accuracy: 0.7975
Epoch 2/100
8000/8000 [==============================] - 0s 62us/sample - loss: 0.1753 - accuracy: 0.7960 - val_loss: 0.1687 - val_accuracy: 0.7975
Epoch 3/100
8000/8000 [==============================] - 0s 61us/sample - loss: 0.1667 - accuracy: 0.7960 - val_loss: 0.1642 - val_accuracy: 0.7975
Epoch 4/100
8000/8000 [==============================] - 0s 60us/sample - loss: 0.1641 - accuracy: 0.7960 - val_loss: 0.1627 - val_accuracy: 0.7975
Epoch 5/100
8000/8000 [==============================] - 0s 60us/sample - loss: 0.1631 - accuracy: 0.7960 - val_loss: 0.1621 - val_accuracy: 0.7975
Epoch 6/100
8000/8000 [==============================] - 0s 62us/sample - loss: 0.1627 - accuracy: 0.7960 - val_loss: 0.1618 - val_accuracy: 0.7975
Epoch 7/100
8000/8000 [==============================] - 0s 61u

### Predict the results using 0.5 as a threshold

In [24]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.20410725]
 [0.2040019 ]
 [0.20410031]
 ...
 [0.20384255]
 [0.20396742]
 [0.20380473]]


In [25]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


### Print the Accuracy score and confusion matrix

In [26]:
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

[[1595    0]
 [ 405    0]]


In [27]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '% of testing data was classified correctly')

79.75 % of testing data was classified correctly


### Optimize the model

Some important parameters to look out for while optimizing neural networks are:

-Type of architecture

-Number of Layers

-Number of Neurons in a layer

-Regularization parameters

-Learning Rate

-Type of optimization / backpropagation technique to use

-Dropout rate

-Weight sharing

#### Number of Layers:
We will keep it similar to the above model so that we can compare the accuracy.
1 hidden layer.

#### Activation:
input layer: relu becasue we are in an input layer. uses the ReLu activation function for  ϕ
output layer: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for  ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

#### Type of optimization / backpropagation technique to use: 
We will use Adam. Adam is a very efficeint variation of Stochastic Gradient Descent. For Adam and its variant, learning rate or the decay rate does not really matter too much.

#### Learning Rate:
default learning rate 0.001.

#### Number of Neurons in a layer:
We will keep it 6 as per our initial calculation above.

#### Weight sharing / kernel_initializer: 
uniform the distribution with which we randomly initialize weights for the nodes in this layer.

#### Loss:
loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

### Rebuilding the model using these optimised parameters

In [0]:
# Initializing the ANN
classifier = Sequential()
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [0]:
# Adding the hidden layer
# Notice that we do not need to specify input dim. 
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

In [0]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [0]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [32]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 138us/sample - loss: 0.5562 - accuracy: 0.7960 - val_loss: 0.4379 - val_accuracy: 0.7975
Epoch 2/100
8000/8000 [==============================] - 1s 69us/sample - loss: 0.4370 - accuracy: 0.7960 - val_loss: 0.4305 - val_accuracy: 0.7975
Epoch 3/100
8000/8000 [==============================] - 1s 75us/sample - loss: 0.4312 - accuracy: 0.7960 - val_loss: 0.4276 - val_accuracy: 0.7975
Epoch 4/100
8000/8000 [==============================] - 1s 67us/sample - loss: 0.4283 - accuracy: 0.7960 - val_loss: 0.4249 - val_accuracy: 0.7975
Epoch 5/100
8000/8000 [==============================] - 0s 62us/sample - loss: 0.4257 - accuracy: 0.7960 - val_loss: 0.4228 - val_accuracy: 0.7975
Epoch 6/100
8000/8000 [==============================] - 0s 61us/sample - loss: 0.4224 - accuracy: 0.7960 - val_loss: 0.4198 - val_accuracy: 0.7975
Epoch 7/100
8000/8000 [==============================] - 0s 62u

### Predict the results using 0.5 as a threshold

In [33]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.21548188]
 [0.3674677 ]
 [0.16310653]
 ...
 [0.22479099]
 [0.15512791]
 [0.12217143]]


In [34]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


### Print the Accuracy score and confusion matrix

In [35]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[1542   53]
 [ 266  139]]


In [36]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '% of testing data was classified correctly')

84.05 % of testing data was classified correctly
